In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [8]:
from integrado import *
%load_ext autoreload
%autoreload 2

In [4]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 19:23:28.040593 :: 0.000637 - Processing item #0 
** Processed 10000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [5]:
index_=Index().computePostingLists(tokens,showProgressEach=10000)
len(index_.postlists.keys())

** Counting TF & DF ...
2020-05-12 19:23:37.210103 :: 0.000988 - Processing item #0 
** Calculating TF-IDF...
2020-05-12 19:23:37.909169 :: 0.700054 - Processing item #0 
** Sorting posting lists...
2020-05-12 19:23:38.389240 :: 1.180125 - Processing item #0 
2020-05-12 19:23:38.499494 :: 1.290379 - Processing item #10000 


In [6]:
ds=index_.o('hoy no me puedo levantar',isText=True)
print(len(ds),ds[:5])
docs=tokens.getDocs(ds)
print(len(docs),docs[:5])

619 [3, 5, 9, 45, 54]
619 [['obvi', 'hoy', 'cumpl', 'anos', 'apag', 'voz'], ['hoy', 'cumpl', 'ano', 'plan', 'des', 'secret', 'gent', 'sabi', 'haci', 'sent', 'bien', 'aunqu', 've', 'tan', 'cercan'], ['amig', 'seguidor', 'tod', 'person', 'respet', 'jueg', 'decision', 'jug', 'pued', 'influ', 'ser', 'president', 'estudiantil', 'hag', 'ayud'], ['pregunt', 'quer', 'hoy'], ['pregunt', 'investig', 'aprend', 'pued']]


In [7]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)

20

Total = 21002
[(5688, 'ds'), (5504, 'docs'), (1064, 'Tokens'), (1064, 'InteractiveShell'), (1064, 'Index'), (1064, 'FastText'), (1064, 'Counter'), (400, 'defaultdict'), (278, '_i7'), (248, '_oh'), (248, 'Out'), (195, '_i1'), (172, '_i6'), (172, '_i'), (167, '_i2'), (144, 'tweet_iterator'), (138, '_iii'), (138, '_i4'), (136, '_ih'), (136, 'In'), (133, '_ii'), (133, '_i5'), (113, '__doc__'), (88, 'tb'), (88, 'sys'), (88, 'pd'), (88, 'os'), (88, 'np'), (88, 'gc'), (88, 'datetime'), (88, '__builtins__'), (88, '__builtin__'), (80, '_dh'), (72, 'get_ipython'), (72, '_i3'), (64, 'tokens'), (64, 'quit'), (64, 'index_'), (64, 'exit'), (57, '__name__'), (53, '___'), (53, '__'), (28, '_7'), (28, '_'), (16, '__spec__'), (16, '__package__'), (16, '__loader__')]
